In [13]:
import os
import json
import pickle
import pandas as pd
import numpy as np
from sklearn.linear_model import RidgeCV, LogisticRegressionCV
from sklearn.model_selection import StratifiedKFold
from tqdm.notebook import tqdm
from rca import make_binary_scoring, make_multiclass_scoring, process_categorical, best_logistic_solver, checker, k_fold_cross_val

## Loading Data

In [14]:
# Loading dictionary of dtype to embed
with open('../../data/raw/dtype_to_embed.json', 'r') as f:
    dtype_to_embed = json.load(f)
    
brain_behav_names = dtype_to_embed['brain'] + dtype_to_embed['behavior']

# Iterating through pulled_embeds and finding union of all brain and behavior vocabs
embeds_path = '../../data/raw/embeds/'
brain_behav_union = set()
for name in tqdm(brain_behav_names):
    vocab = set(pd.read_csv(embeds_path + name + '.csv', index_col=0).index)
    brain_behav_union = brain_behav_union.union(vocab)

len(brain_behav_union)  

  0%|          | 0/16 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
# Pulling and standardising embeddings
embeds = {}
embeds_path = '../../data/raw/embeds/'
for f_name in tqdm(os.listdir(embeds_path)):
    if f_name != 'feature_overlap.csv':  # dropping since contains many NaNs
        
        embed = pd.read_csv(embeds_path + f_name, index_col=0)
        embed_name = f_name.split('.')[0]
        
        # Subsetting to brain and behavior vocab
        embed = embed.loc[embed.index.intersection(brain_behav_union)]
        
        # Standardising
        embeds[embed_name] = (embed - embed.mean()) / embed.std()

{name: embed.shape for name, embed in embeds.items()}

In [15]:
norms = pd.read_csv('../../data/raw/psychNorms.zip', index_col=0, compression='zip', low_memory=False)
norm_metadata = pd.read_csv('../../data/raw/psychNorms_metadata.csv', index_col='norm')
norm_metadata['associated_embed'] = norm_metadata['associated_embed'].astype(str)
norms

,Freq_HAL,Freq_KF,Freq_SUBTLEXUS,Freq_SUBTLEXUK,Freq_Blog,Freq_Twitter,Freq_News,Freq_CobW,Freq_CobS,CD_SUBTLEXUS,...,reproduction_vanarsdall,person_vanarsdall,goals_vanarsdall,movement_vanarsdall,concreteness_vanarsdall,familiarity_vanarsdall,imageability_vanarsdall,familiarity_fear,aoa_fear,imageability_fear
'em,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.3617,1.9138,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'neath,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,0.0000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
're,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.9031,1.6335,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'shun,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,0.0000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'tis,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.4771,0.6021,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
shrick,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.62,4.38,2.93
post office,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.79,3.07,5.29
fishing rod,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.29,3.38,5.64
March,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.43,2.76,3.50


In [5]:
# Log transforming selected norms
with open('../../data/processed/norms_to_log.pkl', 'rb') as f:
    norms_to_log = pickle.load(f)
    norms[norms_to_log] = norms[norms_to_log].apply(np.log1p)

with open('../../data/raw/embed_to_dtype.json', 'r') as f:
    embed_to_type = json.load(f)
embed_to_type

{'CBOW_GoogleNews': 'text',
 'fastText_CommonCrawl': 'text',
 'fastText_Wiki_News': 'text',
 'fastTextSub_OpenSub': 'text',
 'GloVe_CommonCrawl': 'text',
 'GloVe_Twitter': 'text',
 'GloVe_Wikipedia': 'text',
 'LexVec_CommonCrawl': 'text',
 'morphoNLM': 'text',
 'spherical_text_Wikipedia': 'text',
 'eye_tracking': 'brain',
 'EEG_speech': 'brain',
 'EEG_text': 'brain',
 'fMRI_speech_hyper_align': 'brain',
 'fMRI_text_hyper_align': 'brain',
 'microarray': 'brain',
 'PPMI_SVD_SWOW': 'behavior',
 'SGSoftMaxInput_SWOW': 'behavior',
 'SGSoftMaxOutput_SWOW': 'behavior',
 'PPMI_SVD_SouthFlorida': 'behavior',
 'PPMI_SVD_EAT': 'behavior',
 'THINGS': 'behavior',
 'feature_overlap': 'behavior',
 'norms_sensorimotor': 'behavior',
 'compo_attribs': 'behavior',
 'SVD_sim_rel': 'behavior'}

## Cross Validation

In [6]:
# Ridge
alphas = np.logspace(-3, 3, 6)
ridge = RidgeCV(alphas=alphas)

# Logistic hyperparameters
Cs = 1 / alphas
inner_cv = 5
penalty = 'l2'

# Scorers
binary_scoring = make_binary_scoring()
multiclass_scoring = make_multiclass_scoring()
continuous_scoring = {'r2': 'r2', 'neg_mse': 'neg_mean_squared_error'}

# outer_cv setting 
outer_cv, n_jobs = 5, 10

In [7]:
# RCA
rca = []
for embed_name in tqdm(embeds.keys()):
    embed = embeds[embed_name]
    
    to_print = []
    for norm_name in tqdm(norms.columns, desc=embed_name):
        
        # Aligning data
        y = norms[norm_name].dropna()
        X, y = embed.align(y, axis=0, join='inner', copy=True)
        
        # Checking norm dtype 
        norm_dtype = norm_metadata.loc[norm_name, 'type']
        
        # Solvers, scoring, estimators
        if norm_dtype in ['binary', 'multiclass']:
            X, y = process_categorical(outer_cv, inner_cv, X, y)
            
            # may have switched form multi to bin after processing
            norm_dtype = 'binary' if len(y.unique()) == 2 else 'multiclass'
            
            # Cross validation settings for logistic regression
            solver = best_logistic_solver(X, norm_dtype)
            
            # Defining logistic regression 
            estimator = LogisticRegressionCV(
                Cs=Cs, penalty=penalty, cv=StratifiedKFold(inner_cv),
                solver=solver, n_jobs=8
            )
            scoring = binary_scoring if norm_dtype == 'binary' else multiclass_scoring
        else: # continuous
            estimator, scoring = ridge, continuous_scoring
  
        # Cross validation
        associated_embed = norm_metadata.loc[norm_name, 'associated_embed']
        check = checker(embed_name, y, norm_dtype, associated_embed, outer_cv)
        if check == 'pass':
             scores = k_fold_cross_val(estimator, X, y, outer_cv, scoring, n_jobs)
             r2s, mses = scores['test_r2'], - scores['test_neg_mse']
             r2_mean, r2_sd = r2s.mean(), r2s.std()
             mse_mean, mse_sd = mses.mean(), mses.std()
        else:
            r2_mean, r2_sd = np.nan, np.nan
            mse_mean, mse_sd = np.nan, np.nan
            
        # Saving
        train_n = int(((outer_cv - 1) / outer_cv) * len(X))
        test_n = len(X) - train_n
        p = X.shape[1]
        embed_type = embed_to_type[embed_name]
        rca.append([
            embed_name, embed_type, norm_name, train_n, test_n, p, 
            r2_mean, r2_sd, mse_mean, mse_sd, check
        ])
        
        to_print.append([norm_name, train_n, r2_mean, r2_sd, check])

    to_print = pd.DataFrame(to_print, columns=['norm' , 'train_n', 'r2_mean', 'r2_sd', 'check'])
    print(to_print.sort_values('r2_mean', ascending=False).head(10))

rca = pd.DataFrame(
    rca, columns=[
        'embed', 'embed_type', 'norm', 'train_n', 'test_n', 'p', 
        'r2_mean', 'r2_sd', 'mse_mean', 'mse_sd', 'check'
    ]
)

rca.to_csv('../../data/final/rca.csv', index=False)
rca

  0%|          | 0/25 [00:00<?, ?it/s]

CBOW_GoogleNews:   0%|          | 0/292 [00:00<?, ?it/s]

/datapool-1/homepoint/zhussain/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/datapool-1/homepoint/zhussain/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/datapool-1/homepoint/zhussain/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/datapool-1/homepoint/zhussain/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/datapool-1/homepoint/zhussain/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not conver

                        norm  train_n   r2_mean     r2_sd check
283        person_vanarsdall      960  0.808290  0.024645  pass
281       thought_vanarsdall      960  0.806842  0.020711  pass
284         goals_vanarsdall      960  0.796911  0.022164  pass
218            valence_britz      375  0.795997  0.024787  pass
219         social_des_britz      375  0.789863  0.024555  pass
162          goals_wilkowski      837  0.785734  0.012998  pass
24              Conc_Glasgow     3672  0.781250  0.009689  pass
177      concreteness_hollis      828  0.778491  0.019184  pass
282  reproduction_vanarsdall      960  0.776782  0.010919  pass
280        living_vanarsdall      960  0.775123  0.012784  pass


PPMI_SVD_SouthFlorida:   0%|          | 0/292 [00:00<?, ?it/s]

...

                      norm  train_n   r2_mean     r2_sd check
218          valence_britz      384  0.825621  0.093826  pass
219       social_des_britz      384  0.819687  0.083322  pass
283      person_vanarsdall      960  0.815314  0.028712  pass
281     thought_vanarsdall      960  0.807988  0.020557  pass
284       goals_vanarsdall      960  0.804642  0.020717  pass
213  tabooness_janschewitz      359  0.798288  0.043338  pass
162        goals_wilkowski      842  0.763279  0.098425  pass
255   likableness_chandler      644  0.755676  0.083720  pass
175         valence_hollis      829  0.740462  0.068868  pass
280      living_vanarsdall      960  0.739901  0.025244  pass


THINGS:   0%|          | 0/292 [00:00<?, ?it/s]

/datapool-1/homepoint/zhussain/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/datapool-1/homepoint/zhussain/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/datapool-1/homepoint/zhussain/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/datapool-1/homepoint/zhussain/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/datapool-1/homepoint/zhussain/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not conver

                        norm  train_n   r2_mean     r2_sd check
105         Biomotion_Binder      128  0.867574  0.038378  pass
280        living_vanarsdall      376  0.843066  0.018136  pass
282  reproduction_vanarsdall      376  0.828345  0.025951  pass
35            Gustatory_Lanc     1235  0.813892  0.040897  pass
111              Body_Binder      128  0.797362  0.067834  pass
124             Taste_Binder      128  0.793488  0.107142  pass
122             Music_Binder      128  0.758826  0.076604  pass
104            Motion_Binder      128  0.726740  0.025597  pass
131              Path_Binder      128  0.725024  0.064865  pass
281       thought_vanarsdall      376  0.724527  0.083121  pass


,embed,embed_type,norm,train_n,test_n,p,r2_mean,r2_sd,mse_mean,mse_sd,check
0,CBOW_GoogleNews,text,Freq_HAL,28012,7003,300,0.522106,0.008390,2.715519,0.072449,pass
1,CBOW_GoogleNews,text,Freq_KF,19285,4822,300,0.500385,0.009733,0.156678,0.004765,pass
2,CBOW_GoogleNews,text,Freq_SUBTLEXUS,28636,7159,300,0.537246,0.009834,0.361360,0.007800,pass
3,CBOW_GoogleNews,text,Freq_SUBTLEXUK,29316,7330,300,0.545626,0.008433,0.446059,0.009071,pass
4,CBOW_GoogleNews,text,Freq_Blog,31876,7969,300,0.523688,0.008467,0.400176,0.009336,pass
...,...,...,...,...,...,...,...,...,...,...,...
7295,THINGS,behavior,familiarity_vanarsdall,376,95,49,0.060692,0.083942,3326.617511,620.203961,pass
7296,THINGS,behavior,imageability_vanarsdall,376,95,49,0.053593,0.095680,1345.018025,292.430129,pass
7297,THINGS,behavior,familiarity_fear,173,44,49,0.139160,0.160894,0.790788,0.216911,pass
7298,THINGS,behavior,aoa_fear,173,44,49,-0.021206,0.121789,0.533727,0.039008,pass
